In [89]:
"""
скрипт для разбиения по тепловозниками и электровозникам
"""

'\nскрипт для разбиения по тепловозниками и электровозникам\n'

In [90]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
pd.options.mode.chained_assignment = None  # default='warn'

In [91]:
path_to_machine = 'data/Машинисты.xlsx'
path_to_reit = 'data/Рейтинг машинисты.xlsx'
path_to_person = 'data/Абитуриенты.xlsx'
path_to_end_folder ='data'

In [92]:
machine_df = pd.read_excel(path_to_machine) # распредление по специальностям
reit_df = pd.read_excel(path_to_reit,skiprows=4,header=None) # файл с таблицей из ворда
df_person = pd.read_excel(path_to_person,sheet_name='Абитуриенты',skiprows=8) # данные абитуриентов

In [93]:
# создаем два датафрейма для электровозников и тепловозников
teplo_df = machine_df[['тепловоз','атт.1','мед справка.1']]
teplo_df.dropna(axis=0,inplace=True)

elect_df = machine_df[['электровоз','атт','мед справка']]

In [94]:
reit_df.head(2)

,0,1,2,3,4,5,6,7,8,9
0,1,154-483-839-91,1,4.85,0,4.847,0,NaN,0,нет
1,2,191-409-787-91,1,4.39,0,4.389,0,NaN,0,да


In [95]:
reit_df.shape

(148, 10)

In [96]:
snils_df = df_person[['ФИО','СНИЛС']]

In [97]:
snils_df = snils_df.drop_duplicates(subset='ФИО') # убираем дубликаты

In [98]:
snils_df.head(2)

,ФИО,СНИЛС
0,Абросова Анастасия Витальевна,147-022-734-38
1,Авдеев Сергей Александрович,155-320-704-36


In [99]:
union_df = snils_df.merge(reit_df,how='outer',left_on='СНИЛС',right_on=1,indicator=True) # объединяем датафреймы

In [100]:
union_df.head(2)

,ФИО,СНИЛС,0,1,2,3,4,5,6,7,8,9,_merge
0,Абросова Анастасия Витальевна,147-022-734-38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,Авдеев Сергей Александрович,155-320-704-36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [101]:
union_df[union_df['_merge'] == 'right_only'].to_excel(f'{path_to_end_folder}/Отсутствует снилс.xlsx') # сохраяем тех у кого нет снилса

In [102]:
clean_reit = union_df[union_df['_merge'] =='both'] # отбираем тех кто есть в обоих датафреймах

In [103]:
clean_reit= clean_reit[['ФИО','СНИЛС',2,5]]


In [104]:
clean_reit.head(2)

,ФИО,СНИЛС,2,5
2,Агафонов Семен Владимирович,172-941-745-90,1.0,3.632
4,Айхлер Павел Владимирович,150-435-730-34,1.0,3.476


In [106]:
teplo_df.head(2)

,тепловоз,атт.1,мед справка.1
0,Колоколов Александр Витальевич,да,нет
1,Воронов Максим Алексеевич,да,нет


In [107]:
elect_df.head(2)

,электровоз,атт,мед справка
0,Арьяев Денис Александрович,да,да
1,Данзанов Баян-Мунхэ Жаргалович,да,да


In [108]:
raw_teplo_df =clean_reit.merge(teplo_df,how='outer',left_on='ФИО',right_on='тепловоз',indicator=True)

In [109]:
zabr_df_teplo=raw_teplo_df[raw_teplo_df['_merge'] == 'right_only'] # те кто забрал документы

In [110]:
# электровозы
raw_electo_df =clean_reit.merge(elect_df,how='outer',left_on='ФИО',right_on='электровоз',indicator=True)
zabr_df_electo=raw_electo_df[raw_electo_df['_merge'] == 'right_only'] # те кто забрал документы

In [111]:
clean_teplo_df = raw_teplo_df[raw_teplo_df['_merge']=='both'] # готовим итоговыйй тепловозник
clean_teplo_df.drop(columns=['тепловоз','_merge'],inplace=True)

In [112]:
clean_teplo_df.head(1)

,ФИО,СНИЛС,2,5,атт.1,мед справка.1
0,Агафонов Семен Владимирович,172-941-745-90,1.0,3.632,да,нет


In [114]:
clean_teplo_df.columns = ['ФИО','СНИЛС','Приоритет','Средний балл','Сдан оригинал','Мед.справка']

In [120]:
clean_teplo_df['Приоритет'] = clean_teplo_df['Приоритет'].astype(int)
clean_teplo_df.sort_values(by='Средний балл',ascending=False,inplace=True) # сортируем по убыванию

In [121]:
clean_teplo_df.head()

,ФИО,СНИЛС,Приоритет,Средний балл,Сдан оригинал,Мед.справка
111,Селиванов Максим Дмитриевич,173-248-343-69,1,4.316,да,да
122,Тиунов Вячеслав Романович,149-109-028-57,1,4.167,да,да
124,Толстиков Даниил Константинович,153-684-128-76,1,4.000,да,да
91,Михайлов Агван Станиславович,146-383-727-84,1,3.905,да,да
54,Загузин Александр Павлович,164-465-018-68,1,3.850,да,да


In [122]:
# готовим итоговый электровозник
clean_electo_df = raw_electo_df[raw_electo_df['_merge']=='both'] # готовим итоговыйй тепловозник
clean_electo_df.drop(columns=['электровоз','_merge'],inplace=True)

In [123]:
clean_electo_df.head(1)

,ФИО,СНИЛС,2,5,атт,мед справка
4,Арьяев Денис Александрович,156-433-144-56,1.0,3.5,да,да


In [124]:
clean_electo_df.columns = ['ФИО','СНИЛС','Приоритет','Средний балл','Сдан оригинал','Мед.справка']

clean_electo_df['Приоритет'] = clean_electo_df['Приоритет'].astype(int)
clean_electo_df.sort_values(by='Средний балл',ascending=False,inplace=True) # сортируем по убыванию



In [125]:
clean_electo_df.head()

,ФИО,СНИЛС,Приоритет,Средний балл,Сдан оригинал,Мед.справка
101,Попов Богдан Сергеевич,191-409-787-91,1,4.389,да,да
114,Скуратов Егор Викторович,146-115-095-36,1,4.333,да,да
86,Маслов Илья Леонидович,195-583-275-21,1,4.250,нет,нет
77,Лемешев Артем Олегович,146-457-910-88,1,4.158,да,да
76,Латкин Максим Николаевич,165-118-759-74,1,4.150,да,нет
